# 1. Install the Required libraries

In [1]:
%pip install semantic-kernel==0.9.1b1

     -------------------------------------- 246.2/246.2 kB 5.0 MB/s eta 0:00:00
     -------------------------------------- 365.3/365.3 kB 4.6 MB/s eta 0:00:00
     ---------------------------------------- 74.3/74.3 kB ? eta 0:00:00
     ---------------------------------------- 82.4/82.4 kB 4.5 MB/s eta 0:00:00
     ---------------------------------------- 60.8/60.8 kB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 50.5/50.5 kB 2.7 MB/s eta 0:00:00
     ---------------------------------------- 76.7/76.7 kB 4.2 MB/s eta 0:00:00
     -------------------------------------- 472.9/472.9 kB 9.8 MB/s eta 0:00:00
  Using cached isodate-0.6.1-py2.py3-none-any.whl (41 kB)
     ---------------------------------------- 85.5/85.5 kB ? eta 0:00:00
     ---------------------------------------- 57.0/57.0 kB 3.1 MB/s eta 0:00:00
     ------------------------------------- 226.7/226.7 kB 13.5 MB/s eta 0:00:00
     ------------------------------------- 199.4/199.4 kB 11.8 MB/s eta 0:00

ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\Python311\\Scripts\\chardetect.exe' -> 'c:\\Python311\\Scripts\\chardetect.exe.deleteme'


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


# 2. Create your environment variables .env file

Add your environment variables then run the cell to create the *.env* file with your environment variable.

# 3. Load the environment variables

In [1]:
# load the environment variables file
from dotenv import load_dotenv
import os
import openai

# Environment variable obtained from Azure Cosmos DB for MongoDB vCore
AZCOSMOS_CONNSTR=os.getenv("COSMOS_DB_CONNSTRING")
# Environment variables you set to be used by the code
AZCOSMOS_API="mongo-vcore" # currently, semantic kernel only supports vCore
AZCOSMOS_DATABASE_NAME=""
AZCOSMOS_CONTAINER_NAME=""

openai.api_type = "azure"
openai.api_key = os.getenv("AZURE_OPENAI_KEY") 
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT") 
openai.api_deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT")
openai.api_embeddings_deployment_name = os.getenv("AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT")
openai.api_version = "2023-07-01-preview"

True

In [5]:
from azure.cosmos import MongoClient, PartitionKey, exceptions
import os
# Initialize Cosmos Client
connection_string = os.getenv('COSMOS_DB_CONNSTRING')  # replace with your Cosmos DB connection string
client = MongoClient(connection_string)

# Create a database
database_name = 'database1'  # replace with your database name
try:
    database = client.create_database(database_name)
except exceptions.CsosmosResourceExistsError:
    database = client.get_database_client(database_name)

# Create a container
container_name = 'container1'  # replace with your container name
try:
    container = database.create_container(id=container_name, partition_key=PartitionKey(path="/partitionKey"))
except exceptions.CosmosResourceExistsError:
    container = database.get_container_client(container_name)

TypeError: CosmosClient.__init__() missing 1 required positional argument: 'credential'

Some of the parameters needed by [Azure Cosmos DB for MongoDB vCore](https://learn.microsoft.com/azure/cosmos-db/mongodb/vcore/vector-search) to create the vector search index are handled by semantic kernel.

In this guide, we are using `text-embedding-ada-002` embedding model to generate the embeddings which uses a 1536-dimensional embedding vector.

The `num_lists` is an integer that represents of clusters that the inverted file (IVF) index uses to group the vector data.

The `similarity` used with IVF index here is the `COS` (cosine distance) but you can also try `L2` (Euclidean distance), and `IP` (inner product). For more information see the [Understand embeddings in Azure OpenAI Service article](https://learn.microsoft.com/azure/ai-services/openai/concepts/understand-embeddings#cosine-similarity).

In [2]:
# collection name will be used multiple times in the code so we store it in a variable
collection_name = os.environ.get("AZCOSMOS_CONTAINER_NAME")

# Vector search index parameters
index_name = "VectorSearchIndex"
vector_dimensions = (
    1536  # text-embedding-ada-002 uses a 1536-dimensional embedding vector
)
num_lists = 1
similarity = "COS"  # cosine distance

# 4. Create Helper Functions

This function takes in a json file of NoSQL records and checks if your data exists in the database using the id of the record then skips the record if it exists or generates embeddings and uploads the database record along with it's embedding.

The `save_information` function does two things: generate embeddings + upload the data to your database.

Learn more about the semantic kernel memory store [here](https://learn.microsoft.com/semantic-kernel/memories/) and the embeddings [here](https://learn.microsoft.com/semantic-kernel/memories/embeddings).

In [3]:
import json
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory
from semantic_kernel.memory.memory_store_base import MemoryStoreBase


async def upsert_data_to_memory_store(
    memory: SemanticTextMemory, store: MemoryStoreBase, data_file_path: str
) -> None:
    """
    This asynchronous function takes two memory stores and a data file path as arguments.
    It is designed to upsert (update or insert) data into the memory stores from the data file.

    Args:
        kernel_memory_store (callable): A callable object that represents the kernel memory store where data will be upserted.
        memory_store (callable): A callable object that represents the memory store where data will be upserted.
        data_file_path (str): The path to the data file that contains the data to be upserted.

    Returns:
        None. The function performs an operation that modifies the memory stores in-place.
    """
    with open(file=data_file_path, mode="r", encoding="utf-8") as f:
        data = json.load(f)
        n = 0
        for item in data:
            n += 1
            # check if the item already exists in the memory store
            # if the id doesn't exist, it throws an exception
            try:
                already_created = bool(
                    await store.get(
                        collection_name, item["id"], with_embedding=True
                    )
                )
            except Exception:
                already_created = False
            # if the record doesn't exist, we generate embeddings and save it to the database
            if not already_created:
                await memory.save_information(
                    collection=collection_name,
                    id=item["id"],
                    # the embedding is generated from the text field
                    text=item["content"],
                    description=item["title"],
                )
                print(
                    "Generating embeddings and saving new item:",
                    n,
                    "/",
                    len(data),
                    end="\r",
                )
            else:
                print("Skipping item already exits:", n, "/", len(data), end="\r")

# 5. Add the Chat and Embedding models to the Semantic Kernel

Import the semantic kernel, and initialize the semantic kernel.

In [4]:
import semantic_kernel as sk

# Intialize the kernel
kernel = sk.Kernel()

Import the needed libraries.

We need the chat completion for having a conversation and text embeddings for generating embeddings.

In [5]:
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    AzureTextEmbedding,
)

Load the chat deployment name, initialize the chat completions with the required parameters, and add the created chat service to the semantic kernel instance.

In [6]:
# adding azure openai chat service
chat_model_deployment_name = os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
api_key = os.environ.get("AZURE_OPENAI_API_KEY")

kernel.add_service(
    AzureChatCompletion(
        service_id="chat_completion",
        deployment_name=chat_model_deployment_name,
        endpoint=endpoint,
        api_key=api_key,
    )
)
print("Added Azure OpenAI Chat Service...")

Added Azure OpenAI Chat Service...


Load the embeddings deployment name and initialize the text embedding with the required parameters, and add the created embedding service to the semantic kernel instance.

In [7]:
# adding azure openai text embedding service
embedding_model_deployment_name = os.environ.get(
    "AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT_NAME"
)

kernel.add_service(
    AzureTextEmbedding(
        service_id="text_embedding",
        deployment_name=embedding_model_deployment_name,
        endpoint=endpoint,
        api_key=api_key,
    )
)
print("Added Azure OpenAI Embedding Generation Service...")

Added Azure OpenAI Embedding Generation Service...


# 6. Create or Update Azure Cosmos DB for MongoDB

The semantic kernel can handel the database, collection, index creation.

Import the Azure CosmosDB memory store and initialize it with the parameters defined before.

If the database, collection, and index exist it won't overwrite it.

In [8]:
from semantic_kernel.connectors.memory.azure_cosmosdb import (
    AzureCosmosDBMemoryStore,
)

print("Creating or updating Azure Cosmos DB Memory Store...")
# create azure cosmos db for mongo db vcore api store and collection with vector ivf
# currently, semantic kernel only supports the ivf vector kind
store = await AzureCosmosDBMemoryStore.create(
    cosmos_connstr=os.environ.get("AZCOSMOS_CONNSTR"),
    cosmos_api=os.environ.get("AZCOSMOS_API"),
    database_name=os.environ.get("AZCOSMOS_DATABASE_NAME"),
    collection_name=collection_name,
    index_name=index_name,
    vector_dimensions=vector_dimensions,
    num_lists=num_lists,
    similarity=similarity,
)
print("Finished updating Azure Cosmos DB Memory Store...")

Creating or updating Azure Cosmos DB Memory Store...
Finished updating Azure Cosmos DB Memory Store...


Add the created memory store to the semantic kernel instance.

In [9]:
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory
from semantic_kernel.core_plugins.text_memory_plugin import TextMemoryPlugin

memory = SemanticTextMemory(storage=store, embeddings_generator=kernel.get_service("text_embedding"))
kernel.import_plugin_from_object(TextMemoryPlugin(memory), "TextMemoryPluginACDB")
print("Registered Azure Cosmos DB Memory Store...")

Registered Azure Cosmos DB Memory Store...


# 7. Generate embeddings and Create Database records

Call the helper function with the JSON data file to generate embeddings and create or update the database records.

If the records already exit it will skip it.

Records are identified by their ids.

The data used here is a dummy data which you can replace with your own.

**Note that you need to specify id, text, and description fields.
The text field is what gets converted to embeddings.**

See the helper function definition for more information.

In [10]:
print("Upserting data to Azure Cosmos DB Memory Store...")
await upsert_data_to_memory_store(memory, store, "./src/data/text-sample.json")

Upserting data to Azure Cosmos DB Memory Store...


# 8. Test the Vector Database

The search function converts the query_term to a vector embedding and finds the similarity between it and the database records.

In [11]:
# each time it calls the embedding model to generate embeddings from your query
query_term = "What is Azure Database for Managed Instances?"
result = await memory.search(collection_name, query_term)

In [12]:
print(
    f"Result is: {result[0].text}\nRelevance Score: {result[0].relevance}\nFull Record: {result[0].additional_metadata}"
)

Result is: Azure SQL Managed Instance is a fully managed, scalable, and secure SQL Server instance hosted in Azure. It provides features like automatic backups, monitoring, and high availability. SQL Managed Instance supports various data types, such as JSON, spatial, and full-text. You can use Azure SQL Managed Instance to migrate your existing applications, build new applications, and ensure the performance and security of your data. It also integrates with other Azure services, such as Azure App Service and Azure Data Factory.
Relevance Score: 0.8967287909762478
Full Record: {"text": "Azure SQL Managed Instance is a fully managed, scalable, and secure SQL Server instance hosted in Azure. It provides features like automatic backups, monitoring, and high availability. SQL Managed Instance supports various data types, such as JSON, spatial, and full-text. You can use Azure SQL Managed Instance to migrate your existing applications, build new applications, and ensure the performance and

# 9. Create chat function with Azure OpenAI chat model

In [13]:
prompt = """
    You are a chatbot that can have a conversations about any topic related to the provided context.
    Give explicit answers from the provided context or say 'I don't know' if it does not have an answer.
    provided context: {{$db_record}}

    User: {{$query_term}}
    Chatbot:"""

In [14]:
import semantic_kernel.connectors.ai.open_ai as sk_oai

execution_settings = sk_oai.OpenAITextPromptExecutionSettings(
   service_id="chat_completion",
    ai_model_id=chat_model_deployment_name,
    max_tokens=500,
    temperature=0.0,
    top_p=0.5
)

In [15]:
from semantic_kernel.prompt_template.input_variable import InputVariable

chat_prompt_template_config = sk.PromptTemplateConfig(
    template=prompt,
    name="grounded_response",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="db_record", description="The database record", is_required=True),
        InputVariable(name="query_term", description="The user input", is_required=True),
    ],
    execution_settings=execution_settings
)

In [16]:
chat_function = kernel.create_function_from_prompt(
    prompt=prompt,
 function_name= "ChatGPTFunc2", plugin_name="chatGPTPlugin2", prompt_template_config=chat_prompt_template_config
)

In [17]:
completions_result = await kernel.invoke(chat_function, sk.KernelArguments(query_term=query_term, db_record=result[0].additional_metadata))

In [18]:
print(completions_result)

Azure SQL Managed Instance is a fully managed, scalable, and secure SQL Server instance hosted in Azure. It provides features like automatic backups, monitoring, and high availability. SQL Managed Instance supports various data types, such as JSON, spatial, and full-text. You can use Azure SQL Managed Instance to migrate your existing applications, build new applications, and ensure the performance and security of your data. It also integrates with other Azure services, such as Azure App Service and Azure Data Factory.


# 10. Testing the RAG flow 

In [19]:
import time

query_term = ""
while query_term != "exit":
    query_term = input("Enter a query: ")
    result = await memory.search(collection_name, query_term)
    completions_result = kernel.invoke_stream(chat_function, sk.KernelArguments(query_term=query_term, db_record=result[0].additional_metadata))
    print(f"Question:\n{query_term}\nResponse:")
    async for completion in completions_result:
        print(str(completion[0]), end="")
    print("\n")
    time.sleep(5)

Question:
Hey
Response:
Hello! How can I assist you today?

Question:
What is the best place to host my website on Azure?
Response:
Azure App Service is a great option for hosting your website on Azure. It is a fully managed platform that supports a variety of programming languages and frameworks. It also offers built-in auto-scaling and load balancing capabilities, making it easy to scale your website as needed. Additionally, it provides integration with other Azure services, such as Azure DevOps, GitHub, and Bitbucket.

Question:
What is the easiest database to use on Azure?
Response:
Azure SQL Database is the easiest database to use on Azure.

Question:
exit
Response:
Goodbye! Have a great day!

